In [2]:
!pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/cb/7b/2fb6ab376c78d2e60812ad0bf4b24889309d2aeb543f54efcd586c69907f/geopandas-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.19 from https://files.pythonhosted.org/packages/43/56/915f3e4c0016f5917d65f70097e1080f1f5341f3cd2d48d9738c2465fffa/Fiona-1.9.4.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 139.3 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for pyproj>=3.0.1 from https://files.pythonhosted.org/packages/81/eb/3e31e15fdee9d54bdbc575b6384bd7e54f63590fcb4d5c247ad38a81eb44/pyproj-3.6.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 137.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 179.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import geopandas as gpd

# Shapefileのパス
shapefile_path = '~/labs/2023/07/18/002/Akita_City_shp/floodarea.shp'

# GeoDataFrameの作成
gdf_shapefile = gpd.read_file(shapefile_path)

print(gdf_shapefile)

   gridcode         浸水深                                           geometry
0         1      0.5m未満  MULTIPOLYGON (((140.13878 39.70421, 140.13872 ...
1         2  0.5m以上1m未満  MULTIPOLYGON (((140.13842 39.70425, 140.13842 ...
2         3    1m以上2m未満  MULTIPOLYGON (((140.13920 39.70564, 140.13914 ...
3         4        2m以上  MULTIPOLYGON (((140.13453 39.70642, 140.13453 ...


In [7]:
print(gdf_shapefile.dtypes)

gridcode       int64
浸水深           object
geometry    geometry
dtype: object


In [9]:
gdf_shapefile = gdf_shapefile.rename(columns={'浸水深': 'flood_depth'})
print(gdf_shapefile.head())

   gridcode flood_depth                                           geometry
0         1      0.5m未満  MULTIPOLYGON (((140.13878 39.70421, 140.13872 ...
1         2  0.5m以上1m未満  MULTIPOLYGON (((140.13842 39.70425, 140.13842 ...
2         3    1m以上2m未満  MULTIPOLYGON (((140.13920 39.70564, 140.13914 ...
3         4        2m以上  MULTIPOLYGON (((140.13453 39.70642, 140.13453 ...


In [5]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 6.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.6-cp311-cp311-linux_x86_64.whl size=168908 sha256=3faa64792dffe6bbdd3bc8cac4ddbfae2c3680d79456dba1ef05c9a08dbdaab9
  Stored in directory: /home/jovyan/.cache/pip/wheels/a2/c5/3d/b8fd44767c125a321b8d6d7fbcce0181d2f2d1e34967c81365
Successfully built psycopg2


In [11]:
!pip install geoalchemy2

  Obtaining dependency information for geoalchemy2 from https://files.pythonhosted.org/packages/42/6e/0432c606c800923b407235072f8209dfa177b6486513563b41cbcc46f6c9/GeoAlchemy2-0.14.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 1.5 MB/s eta 0:00:00ta 0:00:01


In [1]:
from sqlalchemy import create_engine

# PostgreSQLの情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# GeoDataFrameをPostGISに保存するためのエンジンを作成
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

In [12]:
# GeoDataFrameをPostGISに保存
table_name = 'flood_areas_akita_2023_07_18'
gdf_shapefile.to_postgis(table_name, engine, if_exists='replace')

In [3]:
# 結果を確認

import geopandas as gpd

# テーブルの内容を取得
table_name = 'flood_areas_akita_2023_07_18'
query = f"SELECT * FROM {table_name};"
df = gpd.read_postgis(query, engine, geom_col='geometry')

# 内容の表示
print(df.head())

   gridcode flood_depth                                           geometry
0         1      0.5m未満  MULTIPOLYGON (((140.13878 39.70421, 140.13872 ...
1         2  0.5m以上1m未満  MULTIPOLYGON (((140.13842 39.70425, 140.13842 ...
2         3    1m以上2m未満  MULTIPOLYGON (((140.13920 39.70564, 140.13914 ...
3         4        2m以上  MULTIPOLYGON (((140.13453 39.70642, 140.13453 ...


In [7]:
import psycopg2

# PostgreSQLの接続情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# テーブル名
hospitals_table = 'hospitals_akita_2023_07_18'
flood_areas_table = 'flood_areas_akita_2023_07_18'

# PostgreSQLへの接続
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# カーソルを作成
cursor = conn.cursor()

# 浸水している病院の検索クエリ
query = f"""
SELECT h.id, h.name
FROM {hospitals_table} AS h
WHERE EXISTS (
    SELECT 1
    FROM {flood_areas_table} AS f
    WHERE ST_Intersects(h.geom, f.geometry)
);
"""

# クエリの実行
cursor.execute(query)

# 結果の表示
for row in cursor.fetchall():
    print(row)

# カーソルと接続をクローズ
cursor.close()
conn.close()
